In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

N_SUBJECTS = 339
subjects = range(N_SUBJECTS)
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

In [2]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
  ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids

def remove_omission_errors(HCP_DIR, subjects):
    """Remove omission errors from _bk_err.txt file.

    Args:
      HCP_DIR (str) : Path to target directory
    Returns:
      New files '_bk_err2.txt' in source directory 
  """
    task_keys = ["tfMRI_WM_RL", "tfMRI_WM_LR"]
    errconditions = ["0bk_err", "2bk_err"]
    nlrconditions = ["0bk_nlr", "2bk_nlr"]
    for subject in subjects: 
        for task_key in task_keys:
            #Load error files as numpy arrays 
            nlrconcat = np.zeros([1,3])
            ONSETS = []
            for i in np.arange(2):
                #print('subject {0}, task key {1}, i = {2}'.format(subject, task_key, i))
                errorcondition = errconditions[i]
                nlr = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{nlrconditions[i]}.txt")
                err = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{errorcondition}.txt")
                if nlr.size == 0:
                    err2 = err
                    write_files(err2, subject, task_key, HCP_DIR, errorcondition)
                    continue
                elif nlr.ndim == 1 and err.ndim == 1:
                    onsets = nlr[0]
                    mask = np.isin(err, nlr)[0]
                    err2 = np.delete(err, np.where(mask==True), axis=0)
                    write_files(err2, subject, task_key, HCP_DIR, errorcondition)
                elif nlr.ndim == 1 and err.ndim > 1:
                    onsets = nlr[0]
                    mask = np.isin(err, nlr)[:,0]
                    err2 = np.delete(err, np.where(mask==True), axis=0)
                    write_files(err2, subject, task_key, HCP_DIR, errorcondition)
                else:
                    onsets = nlr[:,0]
                    mask = np.isin(err, nlr)[:,0]
                    err2 = np.delete(err, np.where(mask==True), axis=0)
                    write_files(err2, subject, task_key, HCP_DIR, errorcondition)
                ONSETS = np.append(ONSETS, onsets)

            all_err = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/all_bk_err.txt")
            errorcondition = 'all_bk_err'
            if all_err.size == 0:
                all_err2 = all_err
                write_files(err2, subject, task_key, HCP_DIR, errorcondition )
                continue
            elif all_err.ndim == 1:
                mask = np.isin(all_err[0], ONSETS)
                all_err2 = np.delete(all_err, np.where(mask==True), axis=0)
                write_files(err2, subject, task_key, HCP_DIR, errorcondition)
            else:
                mask = np.isin(all_err[:,0], ONSETS)
                all_err2 = np.delete(all_err, np.where(mask==True), axis=0)
                write_files(err2, subject, task_key, HCP_DIR, errorcondition)

def write_files(err2, subject, task_key, HCP_DIR, errorcondition):
    '''Write txt files from arrays, no matter whether they are empty'''
    # Check for size 
    if err2.size < 3: 
        err2 = []
        np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{errorcondition}2.txt",
               err2, delimiter='\t')
    else: 
        if err2.size == 3:
            err2 = err2.reshape(1,3)
        np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{errorcondition}2.txt",
               err2, fmt=['%.3f','%.1f', '%.d'], delimiter='\t')
        
def count_errors(conditions, subjects, HCP_DIR):
    task = 'wm'
    # Ignore warnings of empty files 
    import warnings
    warnings.filterwarnings("ignore", message="genfromtxt")
    conditions = conditions
    Totalerrorcount = np.zeros([len(subjects), 2, len(conditions)])
    c = -1
    for subject in subjects:
        '''Get number of erroneous trials'''
        i = -1
        for id in get_image_ids(task):
            i += 1
            task_key = BOLD_NAMES[id - 1]
            c = -1 
            for condition in conditions:
                c += 1
                ev_file = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{condition}.txt"
                x = np.genfromtxt(ev_file)
                if x.size == 0: 
                    n_errors = 0
                elif x.size == 3: 
                    n_errors = 1
                else:         
                    n_errors = x.shape[0]
                Totalerrorcount[[subject],[i], [c]] = n_errors

    print(np.sum(Totalerrorcount))
    print(np.sum(Totalerrorcount[:,0,:], axis=0))
    print(np.sum(Totalerrorcount[:,1,:], axis=0))
    
    

In [3]:
#Enter file name 
HCP_DIR = '.'
  
#Calculate N of errors before omission removal  
conditions = ['0bk_err','0bk_nlr', '2bk_err', '2bk_nlr']  
count_errors(subjects=subjects, conditions=conditions, HCP_DIR=HCP_DIR)  

#Remove omissions 
remove_omission_errors(HCP_DIR, subjects)  

#Calculate N of errors after omission removal  
conditions = ['0bk_err2','0bk_nlr', '2bk_err2', '2bk_nlr']
count_errors(subjects=subjects, conditions=conditions, HCP_DIR=HCP_DIR)

7808.0
[1205.  365. 2519.  803.]
[ 788.  191. 1506.  431.]
6018.0
[ 840.  365. 1716.  803.]
[ 597.  191. 1075.  431.]
